In [ ]:
!apt update -qq;
!!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-ubuntu1604.pin
!sudo mv cuda-ubuntu1604.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/10.1/Prod/local_installers/cuda-repo-ubuntu1604-10-1-local-10.1.243-418.87.00_1.0-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1604-10-1-local-10.1.243-418.87.00_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-10-1-local-10.1.243-418.87.00/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda
!/usr/local/cuda/bin/nvcc --version


E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
[sudo] password for tejas: 

In [1]:
code_sum_mean = '''#include <iostream>
#include <cstdio>

using namespace std;


__global__ void sum(int *a, int *b, int n)
{
    int block = 256*blockIdx.x;
    int sum = 0;
    
    for(int i=block;i<min(block+256,n);i++)
    {
        sum = sum + a[i];
    }
    b[blockIdx.x] = sum;
}

int main()
{
    cout<<"Enter no. of elements"<<endl;
    int n;
    cin>>n;
    
    int a[n];
    
    for(int i=0;i<n;i++)
    {
        a[i] = i+1;
    }
    
    int *ad, *bd;
    int size = n*sizeof(int);
    
    cudaMalloc(&ad, size);
    cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
    
    int grids = ceil(n*1.0f/256.0f);
    
    cudaMalloc(&bd, grids*sizeof(int));
    
    dim3 grid(grids, 1);
    dim3 block(1,1);
    
    int p=n;
    
    while(n>1)
    {
        sum<<<grid, block>>>(ad, bd, n);
        n = ceil(n*1.0f/256.0f);
        cudaMemcpy(ad, bd, n*sizeof(int), cudaMemcpyDeviceToHost);
    }
    
    
    int add[2];
    n = p;
    
    cudaMemcpy(add, ad, 4, cudaMemcpyDeviceToHost);
    cout<<"The sum is"<<add[0]<<endl;
    
    float mean=0.0f;
    mean = add[0]/(n*1.0f);
    cout<<"The mean is"<<mean<<endl;
    
    return 0;
}'''

In [2]:
text_file = open("code_sum_mean.cu", "w")
text_file.write(code_sum_mean)
text_file.close()

In [3]:
!nvcc code1.cu

/bin/sh: 1: nvcc: not found


In [15]:
!./a.out

Enter no. of elements
5
The sum is15
The mean is3


In [ ]:
code_min = '''#include <iostream>
#include <cstdio>

using namespace std;
__global__ void min1(int *a, int*b, int n)
{
    int block = 256*blockIdx.x;
    int mini = 7888888;
    
    for(int i=block; i<min(256+block,n);i++)
    {
        if(mini > a[i])
        {
            mini = a[i];
        }
    }
    b[blockIdx.x] = mini;
}

int main()
{
    cout<<"Enter the size of array"<<endl;
    int n;
    cin>>n;
    int a[n];
    
    for(int i=0;i<n;i++)
    {
        a[i] = i+1;
    }
    
    int *ad, *bd;
    int size = n*sizeof(int);
    
    cudaMalloc(&ad, size);
    cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
    
    int grids = ceil(n*1.0f/256.0f);
    cudaMalloc(&bd, grids*sizeof(int));
    
    dim3 grid(grids,1);
    dim3 block(1,1);
    
    while(n>1)
    {
        min1<<<grid, block>>>(ad, bd, n);
        n = ceil(n*1.0f/256.0f);
        cudaMemcpy(ad, bd, n*sizeof(int), cudaMemcpyDeviceToHost);
    }
    
    int ans[2];
    
    cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);
    
    cout<<"The min element is"<<ans[0]<<endl;
    
    return 0;
}'''

In [ ]:
text_file = open("code_min.cu", "w")
text_file.write(code_min)
text_file.close()

In [ ]:
!nvcc code_min.cu

In [20]:
!./a.out

Enter the size of array
7
The min element is1


In [ ]:
code_std = '''#include <iostream>
#include <cstdio>

using namespace std;

__global__ void stdeviation(int *a, float *b, float mean, int n)
{
    int block = blockIdx.x;
    b[0] = 0.0;
    
    for(int i=block; i<n; i++)
    {
        b[0] += (a[i] - mean)*(a[i] - mean);
    }
    b[0] = b[0]/n;
}

int main()
{
    int n;
    cout<<"Enter number of elements"<<endl;
    cin>>n;
    int a[n];
    
    for(int i=0; i<n; i++)
    {
        a[i] = i+1;
    }
    float mean = (n+1)/2;
    int *ad;
    float *bd;
    
    cudaMalloc(&ad, n*sizeof(int));
    cudaMalloc(&bd, sizeof(float));
    
    cudaMemcpy(ad, a, n*sizeof(int), cudaMemcpyHostToDevice);
    stdeviation<<<n, 1>>> (ad, bd, mean, n);
    
    float ans[1];
    cudaMemcpy(ans, bd, sizeof(float), cudaMemcpyDeviceToHost);
    cout<<"Answer is:"<<sqrt(ans[0])<<endl;
    
    return 0;
}'''

In [ ]:
text_file = open("code_std.cu", "w")
text_file.write(code_std)
text_file.close()

In [ ]:
!nvcc code_std.cu

In [27]:
!./a.out

Enter number of elements
5
Answer is:1.41421


In [ ]:
code_vec_add = '''#include <iostream>
#include <chrono>

using namespace std;
using namespace std::chrono;

__global__ void vecAdd(int *a, int *b, int *c, int n)
{
    int block = blockIdx.x;
    if(block<n)
      c[block] = a[block] + b[block];
}

int main()
{
    int n;
    cout<<"Enter no. of elements"<<endl;
    cin>>n;
    int *a = new int[n];
    int *b = new int[n];
    int *c = new int[n];
    
    for(int i=0;i<n;i++)
    {
        a[i] = i+1;
        b[i] = i+1;
    }
    int *ad, *bd, *cd;
    
    cudaMalloc(&ad, n*sizeof(int));
    cudaMemcpy(ad, a, n*sizeof(int), cudaMemcpyHostToDevice);
    
    cudaMalloc(&bd, n*sizeof(int));
    cudaMemcpy(bd, b, n*sizeof(int), cudaMemcpyHostToDevice);
    
    cudaMalloc(&cd, n*sizeof(int));
    
    vecAdd<<<n, 1>>>(ad, bd, cd, n);
    
    cudaMemcpy(c, cd, n*sizeof(int), cudaMemcpyDeviceToHost);
    
    for(int i=0;i<n;i++)
      cout<<c[i]<<endl;
    
    return 0;
}'''

In [ ]:
text_file = open("code_vec_add.cu", "w")
text_file.write(code_vec_add)
text_file.close()

In [ ]:
!nvcc code_vec_add.cu

In [8]:
!./a.out

Enter no. of elements
5
2
4
6
8
10


In [ ]:
code_mat_mul = '''#include <iostream>
using namespace std;
#include<cstdio>

__global__ void matMul(int *a, int *b, int *c, int n)
{
    int row = blockIdx.y*blockDim.y+threadIdx.y;
    int col = blockIdx.x*blockDim.x+threadIdx.x;
    
    int sum = 0;
    
    for(int i=0;i<n;i++)
    {
        sum += a[row*n+i]*b[i*n+col];
    }
    c[n*row+col] = sum;
}

int main()
{
    int n;
    cout<<"Enter n"<<endl;
    cin>>n;
    int *a = new int[n*n];
    int *b = new int[n*n];
    int *c = new int[n*n];
    
    for(int i=0;i<n;i++)
    {
        for(int j=0;j<n;j++)
        {
            a[i*n+j] = i+1;
            b[i*n+j] = j+1;
        }
    }
    
    int *ad, *bd, *cd;
    cudaMalloc(&ad, n*n*sizeof(int));
    cudaMalloc(&bd, n*n*sizeof(int));
    cudaMalloc(&cd, n*n*sizeof(int));
    
    cudaMemcpy(ad, a, n*n*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(bd, b, n*n*sizeof(int), cudaMemcpyHostToDevice);
    
    dim3 grids(n, n, 1);
    
    matMul<<<grids, 1>>>(ad, bd, cd, n);
    
    cudaMemcpy(c, cd, n*n*sizeof(int), cudaMemcpyDeviceToHost);
    
    for(int i=0;i<n;i++)
    {
        for(int j=0;j<n;j++)
        {
            cout<<c[i*n+j]<<" ";
        }
        cout<<endl;
    }
    return 0;
}'''

In [ ]:
text_file = open("code_mat_mul.cu", "w")
text_file.write(code_mat_mul)
text_file.close()

In [ ]:
!nvcc code_mat_mul.cu

In [11]:
!./a.out

Enter n
3
3 6 9 
6 12 18 
9 18 27 
